In [4]:
import tensorflow as tf

In [3]:
import numpy as np

## 1. 损失函数

即预测值与已知答案之间的差距。神经网络模型的效果及优化的目标是通过损失函数来定义的。

损失函数是表示神经网络性能的“恶劣程度”的指标，即当前的神经网络对监督数据在多大程度上不拟合，在多大程度上不一致。而我们神经网络的优化目标就是使得损失函数达到最小，即性能的“恶劣程度”达到最小。

损失函数可以使用任意函数，而比较常用的损失函数有交叉熵误差（cross entropy error，CEE）和均方误差（Mean Squared Error, MSE）。

### 1.1 交叉熵误差

表征的是两个概率分布之间的距离，交叉熵越大说明两个概率分布越远，交叉熵越小说明两个概率分布分布越近，是分类问题中使用较广泛的损失函数。 交叉熵误差损失函数定义如下：

$$H(t, y)=-\sum_{i=1}^{n} t_i \cdot ln y_i$$

其中 $t_i$ 为第 $i$ 个数据的真实值，$y_i$ 为神经网络对第 $i$ 个数据的预测值。

对于多分类问题，神经网络的输出一般不是概率分布，因此需要引入softmax层，使得输出服从概率分布。

TensorFlow API：
 + `tf.keras.losses.categorical_crossentropy `
 + `tf.nn.softmax_cross_entropy_with_logits `
 + `tf.nn.sparse_softmax_cross_entropy_with_logits `
 
也可以直接利用公式 `h = -tf.reduce_sum(t * np.log(y))` 来计算。

**e.g.** 对于二分类问题，已知答案 `t=(1,0)`，现有预测值 `y1=(0.6, 0.4)` 和 `y2=(0.8, 0.2)`，哪个更接近标准答案？

In [6]:
t = np.array([1, 0])
y1 = np.array([0.6, 0.4])
y2 = np.array([0.8, 0.2])

In [14]:
# 使用自己定义的交叉熵误差损失函数
cee = lambda t, y: -tf.reduce_sum(t * np.log(y))
h1 = cee(t, y1)
h2 = cee(t, y2)
print('h1:', h1)
print('h2:', h2)
'h1' if h1 > h2 else 'h2'

h1: tf.Tensor(0.5108256237659907, shape=(), dtype=float64)
h2: tf.Tensor(0.2231435513142097, shape=(), dtype=float64)


'h1'

In [15]:
# 使用 TensorFlow API
tfH1 = tf.losses.categorical_crossentropy(t, y1)
tfH2 = tf.losses.categorical_crossentropy(t, y2)
print('tfH1:', tfH1)
print('tfH2:', tfH2)
'tfH1' if tfH1 > tfH2 else 'tfH2'

tfH1: tf.Tensor(0.5108256237659907, shape=(), dtype=float64)
tfH2: tf.Tensor(0.2231435513142097, shape=(), dtype=float64)


'tfH1'

对比之下验证了使用计算公式和使用 API 的计算结果是一致的

### 1.2 均方误差

均方误差（Mean Square Error）是回归问题最常用的损失函数。回归问题解决的是对具体数值的预测，比如房价预测、销量预测等。这些问题需要预测的不是一个事先定义好的类别，而是一个任意实数。均方误差损失函数定义如下：

$$MSE(y,t)=\frac{ \sum_{i=1}^{n}(y_i - t_i)^2}{n}$$

其中 $y_i$ 为神经网络的对第 $i$ 个数据的预测值，$t_i$ 为第 $i$ 个数据的真实值。

TensorFlow API:
 + `tf.keras.losses.MSE`

也可以直接利用公式 `mse = tf.reduce_sum(tf.square(y - t))` 来计算。

**e.g.** 预测某物的日销量 $y$ 和影响日销量的因素 $x_1$ 和 $x_2$ 的关系。

在建模之前，应预先采集的数据有：某物的日销量 $y$ 和影响日销量的因素 $x_1$ 和 $x_2$ ，数据量越大越好（最佳情况是：某物的产量=某物的销售量）

**由于手头没有相关数据**，所以我们自行制造一套数据集：随机生成 $x_1$ 和 $x_2$， 标准答案：$y = x1 + x2$，噪声：$-0.05 \sim +0.05$。拟合可以预测销量的函数。
|
>这里的意思是，先通过随机数和一个关系来制造数据，并引入一定的噪声使得不完全相等，然后使用梯度下降法来预测数据之间的关系，并与我们给定的标准答案相比较，最终目的就是使得预测值与标准答案之间的损失函数值尽可能的小。

In [16]:
# 制造数据集
SEED = 23455
# 生成 [0, 1) 之间的随机数
rdm = np.random.RandomState(seed=SEED)
# 生成 32 组数据，第 0 列表示 x1 的数据，第 1 列表示 x2 的数据
x = rdm.rand(32, 2)
# 噪声 [0,1) / 10 -0.05 ==> [0, 0.1) - 0.05 ==> [-0.05, 0.05)
noise = rdm.rand() / 10 - 0.05
t = [[x1 + x2 + noise] for (x1, x2) in x]
x = tf.cast(x, dtype=tf.float32)

定义超参数

In [17]:
epoch = 15000
lr = 0.002
# 随机初始化权重参数
w1 = tf.Variable(tf.random.normal([2, 1], stddev=1, seed=1))

In [19]:
for i in range(epoch):
    with tf.GradientTape() as tape:
        y = tf.matmul(x, w1)
        loss_mse = tf.reduce_sum(tf.square(y - t))
    grads = tape.gradient(loss_mse, w1)
    w1.assign_sub(lr * grads)
    
    if epoch % 500 == 0:
        print('After {} training steps, w1 is {}'.format(i, w1.numpy()))
print('Final w1 is:', w1.numpy())

After 0 training steps, w1 is [[-0.7559013]
 [ 1.5153551]]
After 1 training steps, w1 is [[-0.7034313]
 [ 1.5431635]]
After 2 training steps, w1 is [[-0.6537145]
 [ 1.5682397]]
After 3 training steps, w1 is [[-0.6065703]
 [ 1.5907841]]
After 4 training steps, w1 is [[-0.5618308]
 [ 1.6109833]]
After 5 training steps, w1 is [[-0.5193395]
 [ 1.6290104]]
After 6 training steps, w1 is [[-0.4789507]
 [ 1.6450267]]
After 7 training steps, w1 is [[-0.44052875]
 [ 1.6591817 ]]
After 8 training steps, w1 is [[-0.4039473]
 [ 1.6716148]]
After 9 training steps, w1 is [[-0.36908865]
 [ 1.6824553 ]]
After 10 training steps, w1 is [[-0.3358432]
 [ 1.6918234]]
After 11 training steps, w1 is [[-0.3041089]
 [ 1.6998308]]
After 12 training steps, w1 is [[-0.27379054]
 [ 1.7065814 ]]
After 13 training steps, w1 is [[-0.24479955]
 [ 1.7121717 ]]
After 14 training steps, w1 is [[-0.21705338]
 [ 1.7166913 ]]
After 15 training steps, w1 is [[-0.19047508]
 [ 1.7202233 ]]
After 16 training steps, w1 is [[-0.16

After 151 training steps, w1 is [[0.83810776]
 [1.1760709 ]]
After 152 training steps, w1 is [[0.84041846]
 [1.174045  ]]
After 153 training steps, w1 is [[0.84269977]
 [1.1720446 ]]
After 154 training steps, w1 is [[0.84495217]
 [1.1700696 ]]
After 155 training steps, w1 is [[0.84717596]
 [1.1681195 ]]
After 156 training steps, w1 is [[0.8493715]
 [1.1661942]]
After 157 training steps, w1 is [[0.8515392]
 [1.1642933]]
After 158 training steps, w1 is [[0.85367936]
 [1.1624163 ]]
After 159 training steps, w1 is [[0.8557924]
 [1.1605632]]
After 160 training steps, w1 is [[0.8578786]
 [1.1587336]]
After 161 training steps, w1 is [[0.8599384]
 [1.1569271]]
After 162 training steps, w1 is [[0.861972 ]
 [1.1551435]]
After 163 training steps, w1 is [[0.8639798]
 [1.1533824]]
After 164 training steps, w1 is [[0.86596215]
 [1.1516438 ]]
After 165 training steps, w1 is [[0.8679193]
 [1.149927 ]]
After 166 training steps, w1 is [[0.8698517]
 [1.1482321]]
After 167 training steps, w1 is [[0.871759

After 307 training steps, w1 is [[0.9954255]
 [1.0380621]]
After 308 training steps, w1 is [[0.9957411]
 [1.0377852]]
After 309 training steps, w1 is [[0.99605274]
 [1.0375117 ]]
After 310 training steps, w1 is [[0.9963604]
 [1.0372417]]
After 311 training steps, w1 is [[0.9966642]
 [1.0369751]]
After 312 training steps, w1 is [[0.99696416]
 [1.036712  ]]
After 313 training steps, w1 is [[0.9972603]
 [1.0364523]]
After 314 training steps, w1 is [[0.99755263]
 [1.0361958 ]]
After 315 training steps, w1 is [[0.9978413]
 [1.0359424]]
After 316 training steps, w1 is [[0.9981263]
 [1.0356923]]
After 317 training steps, w1 is [[0.9984077]
 [1.0354455]]
After 318 training steps, w1 is [[0.99868554]
 [1.0352017 ]]
After 319 training steps, w1 is [[0.99895984]
 [1.034961  ]]
After 320 training steps, w1 is [[0.9992307]
 [1.0347234]]
After 321 training steps, w1 is [[0.99949807]
 [1.0344888 ]]
After 322 training steps, w1 is [[0.99976206]
 [1.0342572 ]]
After 323 training steps, w1 is [[1.000022

 [1.0189139]]
After 472 training steps, w1 is [[1.0172887]
 [1.0188797]]
After 473 training steps, w1 is [[1.0173272]
 [1.0188459]]
After 474 training steps, w1 is [[1.0173651]
 [1.0188127]]
After 475 training steps, w1 is [[1.0174025]
 [1.0187798]]
After 476 training steps, w1 is [[1.0174395]
 [1.0187473]]
After 477 training steps, w1 is [[1.0174761]
 [1.0187153]]
After 478 training steps, w1 is [[1.0175122]
 [1.0186836]]
After 479 training steps, w1 is [[1.0175478]
 [1.0186523]]
After 480 training steps, w1 is [[1.017583 ]
 [1.0186214]]
After 481 training steps, w1 is [[1.0176177]
 [1.0185909]]
After 482 training steps, w1 is [[1.017652 ]
 [1.0185609]]
After 483 training steps, w1 is [[1.0176859]
 [1.0185312]]
After 484 training steps, w1 is [[1.0177193]
 [1.0185019]]
After 485 training steps, w1 is [[1.0177523]
 [1.0184729]]
After 486 training steps, w1 is [[1.0177848]
 [1.0184443]]
After 487 training steps, w1 is [[1.017817]
 [1.018416]]
After 488 training steps, w1 is [[1.0178487]

After 626 training steps, w1 is [[1.019896]
 [1.016592]]
After 627 training steps, w1 is [[1.0199014]
 [1.0165873]]
After 628 training steps, w1 is [[1.0199068]
 [1.0165826]]
After 629 training steps, w1 is [[1.019912]
 [1.016578]]
After 630 training steps, w1 is [[1.0199172]
 [1.0165734]]
After 631 training steps, w1 is [[1.0199224]
 [1.0165689]]
After 632 training steps, w1 is [[1.0199274]
 [1.0165645]]
After 633 training steps, w1 is [[1.0199324]
 [1.0165601]]
After 634 training steps, w1 is [[1.0199373]
 [1.0165558]]
After 635 training steps, w1 is [[1.0199422]
 [1.0165515]]
After 636 training steps, w1 is [[1.0199469]
 [1.0165473]]
After 637 training steps, w1 is [[1.0199517]
 [1.0165431]]
After 638 training steps, w1 is [[1.0199564]
 [1.0165391]]
After 639 training steps, w1 is [[1.019961]
 [1.016535]]
After 640 training steps, w1 is [[1.0199655]
 [1.016531 ]]
After 641 training steps, w1 is [[1.01997 ]
 [1.016527]]
After 642 training steps, w1 is [[1.0199745]
 [1.0165231]]
After

...

 [1.0162231]]
After 14268 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14269 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14270 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14271 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14272 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14273 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14274 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14275 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14276 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14277 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14278 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14279 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14280 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14281 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14282 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14283 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 1428

 [1.0162231]]
After 14425 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14426 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14427 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14428 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14429 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14430 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14431 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14432 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14433 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14434 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14435 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14436 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14437 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14438 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14439 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14440 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 1444

 [1.0162231]]
After 14585 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14586 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14587 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14588 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14589 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14590 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14591 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14592 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14593 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14594 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14595 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14596 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14597 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14598 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14599 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14600 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 1460

 [1.0162231]]
After 14748 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14749 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14750 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14751 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14752 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14753 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14754 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14755 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14756 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14757 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14758 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14759 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14760 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14761 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14762 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14763 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 1476

After 14912 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14913 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14914 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14915 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14916 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14917 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14918 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14919 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14920 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14921 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14922 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14923 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14924 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14925 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14926 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14927 training steps, w1 is [[1.0203166]
 [1.0162231]]
After 14928 training ste

根据拟合结果，可知 $y \approx x_1 + x_2$，与自定义的标准答案 $y = x_1 + x_2$ 基本一致